# ML_PROFILING NOTEBOOK
## It's for doing the ml profiling once the data is analized in DATA_PROFILING NOTEBOOK

In [ ]:

%load_ext autoreload
%autoreload 2
# The %load_ext autoreload and %autoreload 2 magic commands are used to automatically 
# reload modules when they are changed. This can be useful when you are developing code 
# in an interactive environment, as it allows you to see the changes you make to your modules 
# without having to restart the kernel.


    

In [ ]:

# Load the necesary libraries and print the acces to conf file config.yaml

from omegaconf import OmegaConf
import hydra
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
import os


# for global initialization: NOT RECOMMENDED
#initialize(version_base=None, config_path="../src/conf")
#compose(config_name='config')

print("\n\nthis shows the cfg file:")
with initialize(version_base=None, config_path="../src/conf"):
    cfg = compose(config_name='config')
    print(cfg)

import pandas as pd
import numpy as np


In [ ]:
#loading the dataset
data = pd.read_csv(os.path.join('../data/raw', cfg.file_names.raw_file), 
                   encoding=cfg.general_ml.encoding)

In [ ]:
# And finally choose the Ml model to be applied, among: regression, Classifications, time_series, Clustering, NLP
#from pycaret.classification import *

## Findind the most accurate subdataset
### which would be the best models with whole trainning data? 
Once finding, get the three bes and using plot_model with 'feature', find the most reevant variables to use, and cut the entire dataset.


setup the model

In [ ]:
#You can see below the different considerations about model setup, and correct whatever you need to

#selected_models = setup( data = data, 
#                        target = cfg.data_fields.label, session_id=cfg.general_ml.seed,
#                        transformation=True,
#                        fix_imbalance = True, #8:2
#                        normalize=True)

Let`s comapre different models and define the best for this kind of experiment, using F1 (Because Imbalanmce)

In [ ]:
#best_from_selection= compare_models(sort='F1') #F1: because it's an imbalance model

In [ ]:
# In this case, we choose lr, dt and ada as models to use.

In [ ]:
#We need to create feature plots from selected, to check which subset of fields are the most relevant

#selected=['lr', 'dt', 'ada'] # use your own selection
#for sel in selected:
#    print(str(sel)) 
#    plot_model(create_model(sel), plot='feature', use_train_data=True)

In [ ]:
# so seeing the results before.. this is the selection of the best fields:

#best_features = ['dec_o', 'dec', 'like', 'attr', 'from', 'zipcode', 'prob', 'like_o', 'fun', 'undergra', 
#                 cfg.data_fields.label] #this ast one is the target field

## Reshaping the setup with this fields selection and get the new set of bestmodels

In [ ]:
#selected_data = data[best_features]
#selected_data.shape

In [ ]:
# reserving test data

#test_data = selected_data.sample(frac= float(cfg.data_pipeline.data_transform_params.percent_test))
#selected_data = selected_data.drop(test_data.index, axis = 0)
#selected_data.shape, test_data.shape

In [ ]:
# setup mlflow server in s3

# this code never runs standalone, it must be tested in aws

#import mlflow
#mlflow.set_tracking_uri('ec2-3-140-190-151.us-east-2.compute.amazonaws.com:5000')

In [ ]:
# and let's review that the selected fields got the same results

# match_improved = setup(data = selected_data, #see above 
#                        log_experiment = True,
#                        experiment_name = f'{cfg.general_ml.client}-{cfg.general_ml.project}-{cfg.general_ml.experiment}',
#                        target = cfg.data_fields.label, # get the target label from cfg
#                        session_id=cfg.general_ml.seed, # get the seed from config
#                        train_size = 1.0-float(cfg.data_pipeline.data_transform_params.percent_valid), #get %valid from cfg
#                        transformation=True, 
#                        fix_imbalance = True, #8:2
#                        normalize=True)

In [ ]:
# Get the best model

# best_model = compare_models(sort='F1')

In [ ]:
# Ok seems that everything is ok, so print best model

#print (best_model)

In [ ]:
# And create the selected model in ten kfolds

#selected_model = create_model('lr')

In [ ]:
# Now lets find the best hyperparameters for the selected model 

tuned_model = tune_model(selected_model)


In [ ]:
# and .. evaluate the model

plot_model()
evaluate_model(tuned_model)

In [ ]:

# Finally trains a the model on the entire dataset including the hold-out set.

final_model = finalize_model(tuned_model)

In [ ]:
# save the model

save_model(final_model, os.path.join(cfg.paths.models, cfg.file_names.ml_profiling_model))


## Now, once developed a protoype model in this notebook, it`s time to transforming this experiment in a production model. So let's use some different utils tha can serve to this goal


In [ ]:
# Getback the model to make predictions

saved_model = load_model(os.path.join(cfg.paths.models, cfg.file_names.ml_profiling_model));
saved_model

In [ ]:
# making predictions***

#prediction = predict_model(saved_model,data=test_data)

In [ ]:
# Lets print in text the pipeline to be used :

from sklearn import set_config
set_config(display = 'text')
get_config('pipeline')

# ***** be careful to review that this is the whole pipeline, and copy/paste in pipeline.py



In [ ]:
# This print_pycaret_pipeline() function, is useful to identifiy the 
# different commponents of the pipeline and the libraries to be imported
# https://openscoring.io/blog/2023/01/12/converting_sklearn_pycaret3_pipeline_pmml/

from pycaret.internal.preprocess.transformers import TransformerWrapper
from pycaret.internal.pipeline import Pipeline as PyCaretPipeline
try: 
    from sklearn2pmml.util import fqn
except error:
    !pip install sklearn2pmml    
    from sklearn2pmml.util import fqn

def print_pycaret_pipeline(pipeline):
  if not isinstance(pipeline, PyCaretPipeline):
    raise TypeError()
  steps = pipeline.steps
  transformer_steps = steps[:-1]
  final_estimator_step = steps[-1]

  for transformer_step in transformer_steps:
    name = transformer_step[0]
    transformer = transformer_step[1]
    if not isinstance(transformer, TransformerWrapper):
      raise TypeErrpr()
    print("{} -> {} // {} inputs".format(name, fqn(transformer.transformer), len(transformer._include)))
    #put away the len to see the different fields if needed

  name = final_estimator_step[0]
  final_estimator = final_estimator_step[1]
  print("{} -> {}".format(name, fqn(final_estimator)))

print_pycaret_pipeline(saved_model)

In [ ]:
# get_config? 
# set_config?
# to consult/update the different hyperparameters of the experiments
# for example:
get_config('y_transformed')